# Тестирование различных провайдеров LLM

Поддерживаемые провайдеры:
- **Qwen** - локальный/удаленный Qwen API
- **YandexGPT** - Yandex Cloud LLM API
- **ChatGPT** - OpenAI API
- **DeepSeek** - DeepSeek API


In [1]:
import logging
import sys
import time
from pathlib import Path
from typing import Any

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

print(f"📁 Корневая директория проекта: {project_root}")

📁 Корневая директория проекта: /Users/avolon/Documents/SIRIUS/T-bank_NLP_


In [2]:
from tplexity.llm_client import get_llm

print("✅ Импорты выполнены успешно")

✅ Импорты выполнены успешно


In [3]:
TEST_MESSAGES = [
    {
        "role": "system",
        "content": "Ты - полезный ассистент.",
    },
    {
        "role": "user",
        "content": "Привет!",
    },
]

GENERATION_PARAMS = {"temperature": 0.7, "max_tokens": 500}

print("Тестовый запрос:")
print(f"Системный промпт: {TEST_MESSAGES[0]['content']}")
print(f"Сообщение пользователя: {TEST_MESSAGES[1]['content']}")
print(f"Параметры генерации: {GENERATION_PARAMS}")

Тестовый запрос:
Системный промпт: Ты - полезный ассистент.
Сообщение пользователя: Привет!
Параметры генерации: {'temperature': 0.7, 'max_tokens': 500}


In [4]:
async def test_provider(
    provider: str,
    messages: list[dict],
    generation_params: dict,
) -> dict[str, Any]:
    """
    Тестирует работу LLM провайдера на тестовом запросе

    Args:
        provider (str): Название провайдера
        messages (list[dict]): Список сообщений в формате OpenAI (системный промпт + сообщение пользователя)
        generation_params (dict): Параметры генерации

    Returns:
        Словарь с результатами тестирования
    """
    result = {"provider": provider, "status": "unknown", "response": None, "time": 0, "error": None}

    start_time = time.time()

    try:
        print(f"Тестирование провайдера: {provider.upper()}")

        try:
            client = get_llm(provider)
            print(f"✅ Клиент инициализирован: model={client.model}")
        except Exception as e:
            error_msg = f"Ошибка инициализации клиента: {e}"
            result["error"] = error_msg
            result["status"] = "failed"
            print(f"❌ {error_msg}")
            return result

        try:
            response = await client.generate(
                messages=messages,
                temperature=generation_params.get("temperature"),
                max_tokens=generation_params.get("max_tokens"),
            )

            result["response"] = response
            result["status"] = "success"
            result["time"] = time.time() - start_time

            print(f"✅ Успешно за {result['time']:.2f}с")
            print(f"Ответ: {response}")

        except Exception as e:
            result["status"] = "failed"
            result["error"] = str(e)
            result["time"] = time.time() - start_time
            print(f"❌ Ошибка: {e}")

    except Exception as e:
        result["status"] = "failed"
        result["error"] = f"Критическая ошибка: {e}"
        result["time"] = time.time() - start_time
        print(f"❌ Критическая ошибка: {e}")

    return result

In [5]:
PROVIDERS = ["qwen", "yandexgpt", "chatgpt", "deepseek"]

all_results = {}

for provider in PROVIDERS:
    result = await test_provider(
        provider=provider,
        messages=TEST_MESSAGES,
        generation_params=GENERATION_PARAMS,
    )
    all_results[provider] = result
    print()

print("Тестирование завершено")

2025-11-16 16:59:44,749 - tplexity.llm_client.client - INFO - 🔄 [llm_client] Инициализация LLM клиента: model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ, base_url=http://195.209.210.28:8000/v1
2025-11-16 16:59:44,768 - tplexity.llm_client.client - INFO - ✅ [llm_client] LLM клиент инициализирован


Тестирование провайдера: QWEN
✅ Клиент инициализирован: model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ


2025-11-16 16:59:45,146 - httpx - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 16:59:45,155 - tplexity.llm_client.client - INFO - ✅ [llm_client] Ответ получен от LLM
2025-11-16 16:59:45,155 - tplexity.llm_client.client - INFO - 🔄 [llm_client] Инициализация LLM клиента: model=gpt://b1g737e6b62eqjfkjd8l/yandexgpt-lite, base_url=https://llm.api.cloud.yandex.net/v1
2025-11-16 16:59:45,165 - tplexity.llm_client.client - INFO - ✅ [llm_client] LLM клиент инициализирован


✅ Успешно за 0.41с
Ответ: Привет! Как я могу тебе помочь? 😊

Тестирование провайдера: YANDEXGPT
✅ Клиент инициализирован: model=gpt://b1g737e6b62eqjfkjd8l/yandexgpt-lite


2025-11-16 16:59:45,750 - httpx - INFO - HTTP Request: POST https://llm.api.cloud.yandex.net/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 16:59:45,753 - tplexity.llm_client.client - INFO - ✅ [llm_client] Ответ получен от LLM
2025-11-16 16:59:45,753 - tplexity.llm_client.client - INFO - 🔄 [llm_client] Инициализация LLM клиента: model=gpt-4o-mini, base_url=None
2025-11-16 16:59:45,767 - tplexity.llm_client.client - INFO - ✅ [llm_client] LLM клиент инициализирован


✅ Успешно за 0.60с
Ответ: Здравствуйте! Чем я могу вам помочь?

Тестирование провайдера: CHATGPT
✅ Клиент инициализирован: model=gpt-4o-mini


2025-11-16 16:59:45,991 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 403 Forbidden"
2025-11-16 16:59:45,994 - tplexity.llm_client.client - ERROR - ❌ [llm_client] Ошибка при вызове LLM: Error code: 403 - {'error': {'code': 'unsupported_country_region_territory', 'message': 'Country, region, or territory not supported', 'param': None, 'type': 'request_forbidden'}}
2025-11-16 16:59:45,995 - tplexity.llm_client.client - INFO - 🔄 [llm_client] Инициализация LLM клиента: model=deepseek-chat, base_url=https://api.deepseek.com
2025-11-16 16:59:46,005 - tplexity.llm_client.client - INFO - ✅ [llm_client] LLM клиент инициализирован


❌ Ошибка: Error code: 403 - {'error': {'code': 'unsupported_country_region_territory', 'message': 'Country, region, or territory not supported', 'param': None, 'type': 'request_forbidden'}}

Тестирование провайдера: DEEPSEEK
✅ Клиент инициализирован: model=deepseek-chat


2025-11-16 16:59:46,825 - httpx - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-11-16 16:59:48,606 - tplexity.llm_client.client - INFO - ✅ [llm_client] Ответ получен от LLM


✅ Успешно за 2.61с
Ответ: Привет! 😊 Рад тебя видеть! Как у тебя сегодня дела? Чем могу помочь?

Тестирование завершено
